# Testing the efficiency of the TextAnalysis.jl Sentiment Analysis model

### The full code is on [this GitHubGist](https://gist.github.com/fungktt/a826c6e778f9485d4632e555212a1b85#file-sentiment-analysis-jl) file. 

In [4]:
using TextAnalysis

In [5]:
function setup()
    #opening the txt file containing all the information, and automatically closes it when done
    open("amazonreviews/test.ft.txt") do file
        
        #gathering all the data from the txt file
        global mydata = readlines(file)        
    end
    
    #defining the pre-trained model to be used
    global model = SentimentAnalyzer()
end

setup (generic function with 1 method)

In [6]:
setup()

Sentiment Analysis Model Trained on IMDB with a 88587 word corpus

In [7]:
function analysis()
    #defining two counters which will count the number of reviews analysed and number of correct analysis respectively
    iterations = 0
    tp, tn, fp, fn = 0, 0, 0, 0
    
    #for loop for each line in the txt file
    for num in 1:length(mydata)
        
        #try/catch loop to catch out errors, explained later on
        try
            
            #getting the sentiment value from the txt file, 2 = positive, 1 = negative
            sentiment = parse(Int64,string(mydata[num][10])) 
            
            #getting the review as a string
            sd = StringDocument(mydata[num][12:end])
            
            #removing corrupted characters
            remove_corrupt_utf8!(sd)
            
            #plugging the review into the model to get a floating point number ranging from 0-1, 1 = positive, 0 = negative
            value = model(sd)
            
            #adding one to the number of reviews analysed, to be displayed at the end
            iterations = iterations + 1 
            
            #when both are positive (ie. analysed correctly), add one to the number of correct interpretations
            if value > 0.5 && sentiment == 2
                tp = tp + 1
                
            #when both are negative (ie. analysed correctly), add one to the number of correct interpretations
            elseif value < 0.5 && sentiment == 1
                tn = tn + 1
                
            elseif value > 0.5 && sentiment == 1
                fp = fp +1
                
            elseif value < 0.5 && sentiment == 2
                fn = fn +1
                
            end
            
        #catches out the BoundsError problem
        catch
            
            #skips the rest of the iteration to ignore the review and move onto the next (by the for loop)
            continue
        end
    end
    println("True Positive: ", tp)
    println("True Negative: ", tn)
    println("False Positive: ", fp)
    println("False Negative: ", fn)
end

analysis (generic function with 1 method)

In [8]:
analysis()

True Positive: 118276
True Negative: 88827
False Positive: 110979
False Negative: 81621


## Analysis:

[This blog](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226) gave me an insight on the different metrics we can use to measure the efficiency of our sentiment analysis model. <br>
Also, thanks must go to PseudoCodeNerd for introducing this to me! :)

**Accuracy**: the proportion of true results among the total number of cases examined. <br>
    (TP+TN)/(TP+FP+FN+TN) = 207103/399703 = **0.5181422206**
    
**Precision**: the proportion of results that were actually positive among those that were predicted as positive. <br>
    (TP)/(TP+FP) = 118276/229255 = **0.5159145929**
    
**Recall**: the proportion of results that were predicted as positive amone those that were actually positive. <br>
    (TP)/(TP+FN) = 118276/199897 = **0.5916847176**
    
**F1 Score**: a number between 0 and 1 that is the harmonic mean of precision and recall. <br>
    2(Precision * Recall)/(Precision + Recall) = **0.5512079636**